In [1]:
import torch
import numpy as np
from nilearn import plotting
import clinicadl
import pandas as pd
import matplotlib.pyplot as plt

/network/lustre/dtlake01/aramis/users/sasha.collin/miniconda3/envs/venv/lib/python3.7/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
from clinicadl.tools.tsv.data_split import create_split
from clinicadl.tools.deep_learning.data import generate_sampler, return_dataset, MRIDatasetImage, MRIDatasetSlice, get_transforms
from torch.utils.data import DataLoader
from clinicadl.tools.deep_learning.cnn_utils import train, get_criterion, test
from clinicadl.tools.deep_learning.models.random import RandomArchitecture

In [5]:
data_path = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021.tsv'
summary_path = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021_summary.tsv'
data = pd.read_csv(data_path,sep='\t',nrows=10)
summary = pd.read_csv(summary_path,sep='\t',nrows=10)

In [2]:
input_type = ''
network_type = ''
caps_directory = ''
preprocessing = ''
tsv_path = ''

In [5]:
file = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021/subjects/sub-ADNI002S0295/ses-M00/deeplearning_prepare_data/image_based/t1_linear/sub-ADNI002S0295_ses-M00_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_T1w.pt'

In [6]:
ex = torch.load(file)

In [4]:
data.iloc[:,np.array([0,1,2,3])]

,participant_id,session_id,alternative_id_1,sex
0,sub-ADNI002S0295,ses-M00,002_S_0295,M
1,sub-ADNI002S0295,ses-M06,002_S_0295,M
2,sub-ADNI002S0295,ses-M12,002_S_0295,M
3,sub-ADNI002S0295,ses-M24,002_S_0295,M
4,sub-ADNI002S0295,ses-M36,002_S_0295,M
5,sub-ADNI002S0295,ses-M48,002_S_0295,M
6,sub-ADNI002S0295,ses-M18,002_S_0295,M
7,sub-ADNI002S0295,ses-M30,002_S_0295,M
8,sub-ADNI002S0295,ses-M42,002_S_0295,M
9,sub-ADNI002S0295,ses-M54,002_S_0295,M


# Dataframes experiments

In [21]:
df.iloc[:,841:961+1]

,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Background_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Left-Third-Ventricle_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Right-Third-Ventricle_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Left-Fourth-Ventricle_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Right-Fourth-Ventricle_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Left-Accumbens_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Right-Accumbens_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Right-Amygdala_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Left-Amygdala_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Left-Brainstem_intensity,...,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Left-Parietal-Operculum_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Right-Parietal-Operculum_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Left-Postcentral-Gyrus_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Right-Postcentral-Gyrus_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Left-Posterior-Orbital-Gyrus_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Right-Posterior-Orbital-Gyrus_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Left-Planum-Polare_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Right-Planum-Polare_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Left-Precentral-Gyrus_intensity,t1-volume_group-adni2021_atlas-Neuromorphometrics_ROI-Right-Precentral-Gyrus_intensity
0,0.000012,0.022238,0.026282,0.067420,0.072252,0.697025,0.630125,0.722162,0.737054,0.063721,...,0.311531,0.269510,0.220016,0.207981,0.368762,0.376030,0.317614,0.330355,0.210117,0.236484
1,0.000014,0.023829,0.034454,0.075669,0.087672,0.708202,0.652563,0.729168,0.744579,0.067091,...,0.317509,0.265183,0.218278,0.215057,0.396164,0.400992,0.328085,0.341395,0.217124,0.238370
2,0.000011,0.023700,0.027479,0.071072,0.086042,0.730570,0.671474,0.725941,0.724420,0.070452,...,0.312357,0.265512,0.205282,0.197524,0.380171,0.381976,0.316065,0.336998,0.200715,0.221735
3,0.000013,0.022286,0.032442,0.077022,0.091406,0.706062,0.640589,0.714951,0.719226,0.070082,...,0.305032,0.265200,0.212565,0.194092,0.382101,0.386121,0.316391,0.330592,0.208505,0.227132
4,0.000011,0.024827,0.028868,0.075653,0.086342,0.669194,0.599178,0.690332,0.716521,0.069570,...,0.308398,0.261291,0.209775,0.200719,0.385116,0.384911,0.310383,0.324895,0.211822,0.247875
5,0.000017,0.033929,0.033879,0.072445,0.092303,0.636247,0.640363,0.713178,0.731126,0.073697,...,0.313152,0.269494,0.215650,0.202277,0.378195,0.388771,0.311877,0.326660,0.209985,0.247960
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
pipeline_name = 't1-volume'
atlas_id = 'AAL2'
# first_column_index, last_column_index

In [44]:
df_summary.query("df_summary.pipeline_name==pipeline_name & df_summary.atlas_id == atlas_id")

UndefinedVariableError: name 'df_summary' is not defined

# Iterator

In [23]:
class MRIDatasetSlice(MRIDataset):

    def __init__(self, caps_directory, data_file, slice_index=None, preprocessing="t1-linear",
                 train_transformations=None, mri_plane=0, prepare_dl=False,
                 discarded_slices=20, mixed=False, labels=True, all_transformations=None,
                 multi_cohort=False):
        """
        Args:
            caps_directory (string): Directory of all the images.
            data_file (string or DataFrame): Path to the tsv file or DataFrame containing the subject/session list.
            preprocessing (string): Defines the path to the data in CAPS.
            slice_index (int, optional): If a value is given the same slice will be extracted for each image.
                else the dataset will load all the slices possible for one image.
            train_transformations (callable, optional): Optional transform to be applied only on training mode.
            prepare_dl (bool): If true pre-extracted patches will be loaded.
            mri_plane (int): Defines which mri plane is used for slice extraction.
            discarded_slices (int or list): number of slices discarded at the beginning and the end of the image.
                If one single value is given, the same amount is discarded at the beginning and at the end.
            mixed (bool): If True will look for a 'slice_id' column in the input DataFrame to load each slice
                independently.
            labels (bool): If True the diagnosis will be extracted from the given DataFrame.
            all_transformations (callable, options): Optional transform to be applied during training and evaluation.
            multi_cohort (bool): If True caps_directory is the path to a TSV file linking cohort names and paths.
        """
        # Rename MRI plane
        if preprocessing == "shepplogan":
            raise ValueError("Slice mode is not available for preprocessing %s" % preprocessing)
        self.elem_index = slice_index
        self.mri_plane = mri_plane
        self.direction_list = ['sag', 'cor', 'axi']
        if self.mri_plane >= len(self.direction_list):
            raise ValueError(
                "mri_plane value %i > %i" %
                (self.mri_plane, len(
                    self.direction_list)))

        # Manage discarded_slices
        if isinstance(discarded_slices, int):
            discarded_slices = [discarded_slices, discarded_slices]
        if isinstance(discarded_slices, list) and len(discarded_slices) == 1:
            discarded_slices = discarded_slices * 2
        self.discarded_slices = discarded_slices

        if mixed:
            self.elem_index = "mixed"
        else:
            self.elem_index = None

        self.mode = "slice"
        self.prepare_dl = prepare_dl
        super().__init__(caps_directory, data_file, preprocessing,
                         augmentation_transformations=train_transformations, labels=labels,
                         transformations=all_transformations, multi_cohort=multi_cohort)

    def __getitem__(self, idx):
        participant, session, cohort, slice_idx, label = self._get_meta_data(idx)
        slice_idx = slice_idx + self.discarded_slices[0]

        if self.prepare_dl:
            # read the slices directly
            slice_path = path.join(self._get_path(participant, session, cohort, "slice")[0:-7]
                                   + '_axis-%s' % self.direction_list[self.mri_plane]
                                   + '_channel-rgb_slice-%i_T1w.pt' % slice_idx)
            image = torch.load(slice_path)
        else:
            image_path = self._get_path(participant, session, cohort, "image")
            full_image = torch.load(image_path)
            image = self.extract_slice_from_mri(full_image, slice_idx)

        if self.transformations:
            image = self.transformations(image)

        if self.augmentation_transformations and not self.eval_mode:
            image = self.augmentation_transformations(image)

        sample = {'image': image, 'label': label,
                  'participant_id': participant, 'session_id': session,
                  'slice_id': slice_idx}

        return sample

    def num_elem_per_image(self):
        if self.elem_index is not None:
            return 1

        image = self._get_full_image()
        return image.size(self.mri_plane + 1) - \
            self.discarded_slices[0] - self.discarded_slices[1]

NameError: name 'MRIDataset' is not defined

# Train Single CNN

In [4]:
# global parameters
caps_directory = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021/'
batch_size = 1
num_workers = 1
loss = 'default'
optimizer_name = 'Adam'
learning_rate = 1e-4
weight_decay = 1e-4

In [6]:
# fold iterator
fold_iterator = range(2)

# load dataframes
AD = pd.read_csv('subjects/AD.tsv',sep='\t')
CN = pd.read_csv('subjects/CN.tsv',sep='\t')

# split data between training and validation sets
training_df, valid_df = create_split('AD', AD, 'diagnosis',0.2)
df_CN = create_split('CN', CN, 'diagnosis',0.2)
training_df = training_df.append(df_CN[0]).reset_index().iloc[np.array([0,1,2,-1,-2,-3])]
valid_df = valid_df.append(df_CN[1]).reset_index().iloc[np.array([0,1,2,-1,-2,-3])]

# drop index column
training_df.drop(columns = ['index'], inplace=True)
valid_df.drop(columns = ['index'], inplace=True)

train_transforms, all_transforms = get_transforms('image',
                                                  minmaxnormalization=False,
                                                  data_augmentation=None )

DEBUG:root:p_age=0.54, p_sex=0.9493
DEBUG:root:p_age=0.78, p_sex=0.8008
DEBUG:root:p_age=0.91, p_sex=0.9771
INFO:root:Split for diagnosis AD was found after 3 trials
DEBUG:root:p_age=0.81, p_sex=0.9321
INFO:root:Split for diagnosis CN was found after 1 trials


In [11]:
data_train[0]

{'image': tensor([[[ 3.1245e-02,  2.3667e-01,  2.4358e+00,  ...,  0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [ 5.7567e+00,  5.3266e+00,  3.5201e+00,  ..., -0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [ 1.1674e+01,  1.8963e+01,  1.5350e+01,  ...,  0.0000e+00,
            0.0000e+00, -0.0000e+00],
          ...,
          [ 5.4821e-05,  7.3491e-05,  1.5597e-06,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 1.6332e-19,  6.4112e-07,  3.4469e-20,  ...,  0.0000e+00,
            0.0000e+00, -0.0000e+00],
          [ 1.1628e-19, -7.0409e-21, -7.6672e-20,  ...,  0.0000e+00,
           -0.0000e+00, -0.0000e+00]],
 
         [[ 3.1245e-02,  2.3667e-01,  2.4358e+00,  ...,  0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [ 5.7567e+00,  5.3266e+00,  3.5201e+00,  ..., -0.0000e+00,
           -0.0000e+00, -0.0000e+00],
          [ 1.1674e+01,  1.8963e+01,  1.5350e+01,  ...,  0.0000e+00,
            0.0000e+00, -0.0000e+00],
          

In [7]:
# follow structure of ``train_single_cnn``

# dataset iterator
data_train = MRIDatasetSlice(caps_directory, training_df, train_transformations=all_transforms)
data_valid = MRIDatasetSlice(caps_directory, valid_df, train_transformations=all_transforms)
# sampler
train_sampler=generate_sampler(data_train)
# loaders
train_loader = DataLoader(data_train,
                         batch_size=batch_size,
                         sampler=train_sampler,
                         num_workers=num_workers,
                         pin_memory=True)

valid_loader = DataLoader(data_valid,
                         batch_size=batch_size,
                         sampler=train_sampler,
                         num_workers=num_workers,
                         pin_memory=True)

# initialize the model
convolutions = {
    'conv0':{
        'n_conv':1,
        'in_channels':None,
        'out_channels':16,
        'd_reduction': "MaxPooling"
    },
    'conv1':{
        'n_conv':1,
        'in_channels':None,
        'out_channels':32,
        'd_reduction': "MaxPooling"
    },
    'conv2':{
        'n_conv':1,
        'in_channels':None,
        'out_channels':32,
        'd_reduction': "MaxPooling"
    },
    'conv3':{
        'n_conv':1,
        'in_channels':None,
        'out_channels':64,
        'd_reduction': "MaxPooling"
    },
    'conv4':{
        'n_conv':1,
        'in_channels':None,
        'out_channels':128,
        'd_reduction': "MaxPooling"
    }
}
model = RandomArchitecture(convolutions,1,data_train.size)


# Define criterion and optimizer
criterion = get_criterion(loss)
optimizer = getattr(torch.optim, optimizer_name)(filter(lambda x: x.requires_grad, model.parameters()),
                                                   lr=learning_rate,
                                                   weight_decay=weight_decay)

In [8]:
class Options:
    def __init__(self):
        self.beginning_epoch = 0
        self.tolerance = 0
        self.patience = 10
        self.epochs = 10
        self.gpu = True
        self.batch_size = 1

In [9]:
train(model, train_loader, valid_loader, criterion, optimizer,False,'log_dir','model_dir', Options())

INFO:root:Beginning epoch 0.


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/network/lustre/dtlake01/aramis/users/sasha.collin/miniconda3/envs/venv/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1625, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1632, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 5

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/network/lustre/dtlake01/aramis/users/sasha.collin/miniconda3/envs/venv/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 185, in _worker_loop
    data = fetc